In [ ]:
# matplotlib inline plotting
%matplotlib inline
# make inline plotting higher resolution
%config InlineBackend.figure_format ='svg'

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import gmean
import re
from tqdm import tqdm
import statsmodels.api as sm
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from helpers.portfolio_sorts import estimate_portfolio_sorts, last_of_month
from helpers.expand_to_daily import expand_to_daily
from helpers.hml import high_minus_low, construct_portfolio
from helpers.sql import connect_to_db, read_db, vacuum_db, update_database
from helpers.pretty_print import pretty_print_pval

plt.style.use('ggplot')

# Here we import the results of the portfolio sort analysis

Remember that we only do the sorts for one type of the sentiment score (we can potentially vary this)

In [ ]:
import pickle

with open('new_pf_sorts_nw7_sum_sentiment.bin', 'rb') as file:
# with open('new_pf_sorts_nw7_mean_sentiment.bin', 'rb') as file:
    # read information from portfolio sorts results
    estimated_portfolio_sorts = pickle.load(file)

    file.close()

In [ ]:
engine = connect_to_db()

# setting newey west lags to 7 according to Greene (Econometric Analysis, 7th edition, section 20.5.2, p. 960)
# note this is different than the setting in `portfolio_sorts.ipynb`
NW_LAGS: int = 7

# define looking-back period (number of months)
LOOK_BACK: int = 3

# number of obs in a year
YEARLY_BUSINESS_DAYS: int = 250

# set significant digits
SIGNIFICANT_DIGITS: int = 4

# define important dates
START_DATE_MISSING = datetime(year=2010, month=1, day=1)

# Actual data-ranges
START_DATE = datetime(year=2010, month=1, day=4)
END_DATE = datetime(year=2023, month=1, day=1)

# Models
AP_MODELS = {
    "CAPM": ["mkt-rf"],
    "FF3": ["mkt-rf", "smb", "hml"],
    "FF3_C": ["mkt-rf", "smb", "hml", "mom"],
    "FF5": ["mkt-rf", "smb", "hml", "rmw", "cma"],
}

# Select sentiment measures 
SENTMENT_MEASURES = [
    "aggregate_transformed_residuals",
    "politics_transformed_residuals",
    "importance_of_human_intervantion_transformed_residuals",
    "weather_extremes_transformed_residuals",
]

# Splits
SPLITS = [0.1, 0.2]


In [ ]:
# fetch series of business days (will be used to clean E portfolio and factors)
business_days = read_db(engine=engine, statement='select "index", date from returns')

business_days = business_days.set_index("date", drop=True).index


In [ ]:
# fetch data
meta = read_db(
    statement="select * from meta", engine=engine
)
meta = meta.set_index('instrument')
meta.index.name = None

environment = read_db(
    engine=engine, statement='select * from environment', idx_col='date')

emissions = read_db(
    engine=engine, statement='select * from emissions', idx_col='date')

sentiment = read_db(
    statement="select * from climate_sum_ar1", engine=engine, idx_col="date"
)

market_cap = read_db(
    engine=engine, statement="select * from market_cap", idx_col="date"
)

returns = read_db(
    engine=engine, statement="select * from returns", idx_col="date"
)

snp500 = read_db(
    engine=engine, statement="select * from snp", idx_col="date"
).sort_index()

factors = read_db(
    engine=engine, statement="select * from factors", idx_col="date"
)

riskfree = read_db(
    engine=engine, statement="select * from riskfree", idx_col="date"
)


In [ ]:
sentiment.tail()

## Calculate excess returns r_{i}-r_{rf}

In [ ]:
# calculate excess returns
returns = pd.merge(left=returns, right=riskfree, how='left', right_index=True, left_index=True)

returns.tail()

In [ ]:
for col in returns.columns:
    if col == "rf":
        continue

    returns[col] = returns[col] - returns["rf"]

returns = returns.drop(columns=["rf"])

returns.tail()


In [ ]:
# Clean-up S&P data. Here we remove all S&P observations that we
# do NOT have observations for
snp500 = snp500[snp500["ric"].isin(returns.columns)]


In [ ]:
# limiting factor space $X_t$
factors = factors.loc[START_DATE:]

factors

In [ ]:
# expand to daily for market cap. Fill between but don't forward fill
market_cap = expand_to_daily(
    market_cap,
    start_date=START_DATE,
    end_date=END_DATE,
    start_date_missing=START_DATE_MISSING,
    last_valid=True,
)

# expand to daily for stock-returns
returns = expand_to_daily(
    returns,
    start_date=START_DATE,
    end_date=END_DATE,
    start_date_missing=START_DATE_MISSING,
    ffill=True,
    cut_sample=True,
    last_valid=True,
    ffill_limit=None,
)
riskfree = expand_to_daily(
    riskfree,
    start_date=START_DATE,
    end_date=END_DATE,
    start_date_missing=START_DATE_MISSING,
    ffill=True,
    cut_sample=True,
    last_valid=True,
    ffill_limit=None,
)

# expand to daily for ESG scores. Fill one year ahead unless...
environment = expand_to_daily(
    environment,
    start_date=START_DATE,
    end_date=END_DATE,
    start_date_missing=START_DATE_MISSING,
    ffill_limit=365,
)
# we also choose to back-fill ESG data due to limited data in the beginning of the sample
environment = environment.bfill()

emissions = expand_to_daily(
    emissions,
    start_date=START_DATE,
    end_date=END_DATE,
    start_date_missing=START_DATE_MISSING,
    ffill_limit=365,
)
emissions = emissions.bfill()




# drop columns that are all NaN
market_cap = market_cap.dropna(how="all", axis=1)
returns = returns.dropna(how="all", axis=1)

# Fit Portfolio sort test

In [ ]:
def portfolio_sort_test(
    hml: pd.DataFrame,
    hml_name: str,
    split_forward: bool = False,
    split_backward: bool = False,
):
    # Get model type
    model = hml_name.split(":")[1]

    Y = hml.groupby("date").first()["hml_return"]
    Y = Y.reindex(business_days).dropna()

    # select subsample
    if split_forward:
        Y = Y.loc["2014-01-01":]
    if split_backward:
        Y = Y.loc[:"2014-01-01"]


    X = factors[AP_MODELS[model]]
    X = sm.add_constant(X)
    X = X.reindex(Y.index).dropna()

    # estimate time-series regression using Newey West errors
    fit = sm.OLS(endog=Y, exog=X).fit(cov_type="HAC", cov_kwds={"maxlags": NW_LAGS})

    arrays = [
        np.repeat(hml_name.split(":")[1], 3),
        np.repeat(hml_name.split(":")[0], 3),
        ["annual_return", "annual_alpha", "alpha_t"],
    ]
    tuples = list(zip(*arrays))

    index = pd.MultiIndex.from_tuples(tuples)

    results = pd.DataFrame(
        [
            Y.mean() * YEARLY_BUSINESS_DAYS,
            fit.params["const"] * YEARLY_BUSINESS_DAYS,
            fit.tvalues["const"],
        ],
        index=index,
        columns=[hml_name.split(":")[2]],
    )

    return results


In [ ]:
sort_results = []

for sort in estimated_portfolio_sorts.keys():
    # run on split
    sort_results.append(portfolio_sort_test(estimated_portfolio_sorts[sort], sort, split_forward=False, split_backward=False))
    # sort_results.append(portfolio_sort_test(estimated_portfolio_sorts[sort], sort, split_forward=True, split_backward=False))
    # sort_results.append(portfolio_sort_test(estimated_portfolio_sorts[sort], sort, split_forward=False, split_backward=True))


In [ ]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

sort_results_joined = []

for group in chunker(sort_results, len(SENTMENT_MEASURES)):
    
    placeholder = pd.DataFrame(index=group[0].index)

    for j in group:
        placeholder = pd.merge(placeholder, j, left_index=True, right_index=True)
    
    sort_results_joined.append(placeholder)

In [ ]:
sort_results_concat = pd.DataFrame()

for frame in sort_results_joined:
    sort_results_concat = pd.concat([sort_results_concat, frame])


sort_results_concat

In [ ]:
for row in sort_results_concat.loc[
    sort_results_concat.index.get_level_values(2).str.contains("alpha_t")
].index:
    
    sort_results_concat.loc[row] = (
        sort_results_concat.loc[row]
        .apply(
            pretty_print_pval,
            freedom=estimated_portfolio_sorts[
                "0.1:CAPM:aggregate_transformed_residuals"
            ]["date"]
            .unique()
            .shape[0],
            precision=SIGNIFICANT_DIGITS,
        )
        .astype(str)
        .mask(sort_results_concat.loc[row].isna())
    )



In [ ]:
# rename and prettify
sort_results_concat.index = sort_results_concat.index.map(lambda x: (x[0], x[1].replace('0.1', 'Deciles'), x[2]))
sort_results_concat.index = sort_results_concat.index.map(lambda x: (x[0], x[1].replace('0.2', 'Quintiles'), x[2]))


sort_results_concat.index = sort_results_concat.index.map(lambda x: (x[0], x[1], x[2].replace('annual_return', '$\\mathbb{E}\\left[r_{t}^{UCP}\\right]$')))
sort_results_concat.index = sort_results_concat.index.map(lambda x: (x[0], x[1], x[2].replace('annual_alpha', '$\\widehat{\\alpha}$')))
sort_results_concat.index = sort_results_concat.index.map(lambda x: (x[0], x[1], x[2].replace('alpha_t', '$t\\left(\\widehat{\\alpha}\\right)$')))

sort_results_concat.columns = sort_results_concat.columns.map(lambda x: x.replace('_transformed_residuals', ''))
sort_results_concat.columns = sort_results_concat.columns.map(lambda x: x.replace('_', ' '))
sort_results_concat.columns = sort_results_concat.columns.map(lambda x: x.title())


sort_results_concat

In [ ]:
latex = sort_results_concat.to_latex(
    index=True,
    escape=False,
    sparsify=True,
    multirow=True,
    multicolumn=True,
    bold_rows=True,
    multicolumn_format="c",
    float_format=f"{{:.{SIGNIFICANT_DIGITS}f}}".format,
    position="H",
)

latex = re.sub(r"\\(mid|top|bottom)rule", "", latex)

print(latex)


## Plotting average loadings

Plot the time-series of the average loadings within each estimate for top and bottom quantiles.

This is simply a resource to do some descriptive on the data. We will probably need some of this to work futher on the analysis.

In [ ]:
group = (
    estimated_portfolio_sorts["0.1:CAPM:aggregate_transformed_residuals"]
    .groupby(["date", "hml"])["rating"]
    .mean()
    .reset_index()
    .set_index("date")
)

group = group.loc[group["hml"].isin([True, False])]

group = group[["hml", "rating"]]

fig, ax = plt.subplots(figsize=(8, 6))

group.loc[group["hml"] == True].plot(ax=ax, label="Top 20% mean")
group.loc[group["hml"] == False].plot(ax=ax, label="Top 20% mean")

plt.legend()

plt.show()


# Plot returns for top and bottom groups

In [ ]:
names = {
    "0.1": "Deciles",
    "0.2": "Quintiles",
    "aggregate_transformed_residuals": "Aggregate",
    "politics_transformed_residuals": "Politics",
    "weather_extremes_transformed_residuals": "Weather Extremes",
    "importance_of_human_intervantion_transformed_residuals": "Imp. of Human Intervention",
    
    "CAPM": "CAPM",
    "FF3_C": "FFC",
    "FF3": "FF3",
    "FF5": "FF5",

    "bottom": "Bottom",
    "top": "Top",
}


def plot_returns(sorted_dict: dict, decile: bool = True, title: str = None):
    if decile:
        sorted_dict_filtered = {
            key: sorted_dict[key] for key in sorted_dict if re.match("0.1", key)
        }

    if not decile:
        sorted_dict_filtered = {
            key: sorted_dict[key] for key in sorted_dict if re.match("0.2", key)
        }

    # overwrite object
    sorted_dict = sorted_dict_filtered

    fig, axes = plt.subplots(4, 4, figsize=(13, 12), sharex=True, sharey=True)

    for i, (key, df) in enumerate(sorted_dict.items()):
        Y = df.groupby("date").first()[["high_return", "low_return"]]

        # make sure to drop non-business days
        Y = Y.reindex(business_days).dropna()

        # rename columns
        if decile:
            Y = Y.rename(columns={"high_return": "Top 10%", "low_return": "Bottom 10%"})
        else:
            Y = Y.rename(columns={"high_return": "Top 20%", "low_return": "Bottom 20%"})

        # join riskfree rate
        Y = pd.merge(left=Y, right=riskfree, left_index=True, right_index=True)
        for col in Y:
            if col != "rf":
                Y[col] = Y[col] - Y["rf"]
        Y = Y.drop(columns=["rf"])

        # plot excess returns
        ((Y + 1).cumprod() - 1).plot(
            ax=axes[i // 4, i % 4],
            legend=False,
            style=["-", "-"],
            color=["C3", "C0"],
        )

        axes[i // 4, i % 4].set_ylabel("Cumulative (excess) return (%)", fontsize=9)

        axes[i // 4, i % 4].set_title(
            f'{names[key.split(":")[2]]} ({names[key.split(":")[1]]})',
            fontdict={"fontsize": 11},
        )

        # break

    # Add title
    fig.suptitle(title, fontsize=16)

    # add legend
    plt.figlegend(
        loc="lower center",
        fontsize=12,
        ncol=2,
        labelspacing=1,
        bbox_to_anchor=(0.5, -0.04),
        handles=axes[0, 0].get_legend_handles_labels()[0],
    )

    # adjust spacing
    plt.tight_layout()

    if decile:
        fig.savefig(f"plots/deciles_pf_sorts.png", dpi=300, bbox_inches="tight")
    else:
        fig.savefig(f"plots/quintiles_pf_sorts.png", dpi=300, bbox_inches="tight")

    plt.plot()


plot_returns(estimated_portfolio_sorts, decile=False, title="Cumulative excess returns for portfolios sort (quintile breakpoints)")
plot_returns(estimated_portfolio_sorts, decile=True, title="Cumulative excess returns for portfolios sort (decile breakpoints)")


# Transform environment from TS to panel and store in `env_panel`

In [ ]:
def to_panel(df: pd.DataFrame, col_name: str, frequency: str = "D") -> pd.DataFrame:
    output = pd.DataFrame(columns=["date", "snp_ric", col_name])

    for col in tqdm(df.columns):
        series = df[col].copy()
        series = series.reset_index()

        series["snp_ric"] = col
        series = series.rename(columns={"index": "date", col: col_name})

        output = pd.concat([output, series], axis=0)

    return output


emi_panel = to_panel(emissions, col_name="emissions", frequency="D")
env_panel = to_panel(environment, col_name="env", frequency="D")


In [ ]:
sorts = {}

for key, df in tqdm(
    estimated_portfolio_sorts.items(), total=len(estimated_portfolio_sorts)
):
    top_tickers = df.loc[(df["hml"] == True)]
    bottom_tickers = df.loc[~(df["hml"] == True)]

    # join E rating
    top_tickers = pd.merge(
        left=env_panel,
        right=top_tickers,
        left_on=["date", "snp_ric"],
        right_on=["date", "snp_ric"],
    )
    bottom_tickers = pd.merge(
        left=env_panel,
        right=bottom_tickers,
        left_on=["date", "snp_ric"],
        right_on=["date", "snp_ric"],
    )

    # join emissions
    top_tickers = pd.merge(
        left=emi_panel,
        right=top_tickers,
        left_on=["date", "snp_ric"],
        right_on=["date", "snp_ric"],
    )
    bottom_tickers = pd.merge(
        left=emi_panel,
        right=bottom_tickers,
        left_on=["date", "snp_ric"],
        right_on=["date", "snp_ric"],
    )

    # join meta data
    top_tickers = pd.merge(
        left=top_tickers,
        right=meta[["company_common_name", "trbc_industry_name"]],
        left_on=["snp_ric"],
        right_index=True,
    )
    bottom_tickers = pd.merge(
        left=bottom_tickers,
        right=meta[["company_common_name", "trbc_industry_name"]],
        left_on=["snp_ric"],
        right_index=True,
    )

    sorts[f"top:{key}"] = top_tickers.sort_values('date')
    sorts[f"bottom:{key}"] = bottom_tickers.sort_values('date')


In [ ]:
significant_models = [
    '0.1:CAPM:aggregate_transformed_residuals',
    '0.1:CAPM:politics_transformed_residuals',
    '0.1:CAPM:importance_of_human_intervantion_transformed_residuals',
    '0.2:CAPM:aggregate_transformed_residuals',
    '0.2:CAPM:politics_transformed_residuals',
    '0.2:CAPM:importance_of_human_intervantion_transformed_residuals',

    '0.1:FF3:aggregate_transformed_residuals',
    '0.1:FF3:politics_transformed_residuals',
    '0.1:FF3:importance_of_human_intervantion_transformed_residuals',
    '0.2:FF3:aggregate_transformed_residuals',
    '0.2:FF3:politics_transformed_residuals',
    '0.2:FF3:importance_of_human_intervantion_transformed_residuals',
]

In [ ]:
sorts['top:0.2:CAPM:importance_of_human_intervantion_transformed_residuals'].groupby('date')['emissions'].median().plot()
sorts['bottom:0.2:CAPM:importance_of_human_intervantion_transformed_residuals'].groupby('date')['emissions'].median().plot()

In [ ]:
env_ratings = {}

for sig in significant_models:
    for hml in ['top', 'bottom']:
        df = sorts[f'{hml}:{sig}']

        env_ratings[f'{hml}:{sig}'] = {
            'e_rating:mean': df.groupby('date')['env'].mean().mean(),
            'e_rating:std':df.groupby('date')['env'].mean().std(), 
            'emissions:mean': df.groupby('date')['emissions'].mean().mean(),
            'emissions:std': df.groupby('date')['emissions'].mean().std(),
            'emissions:coverage': 1 - (df.set_index('date').isnull().groupby('date')['emissions'].sum() / df.groupby('date')['emissions'].count()).mean(),
            #'emissions:median': df.groupby('date')['emissions'].median().mean(),
        }


env_ratings = pd.DataFrame(env_ratings).T

In [ ]:
env_ratings['name'] = env_ratings.index


index_cols = ['sort', 'decile', 'model', 'sent']
env_ratings[index_cols] = env_ratings['name'].str.split(':', expand=True)

for col in index_cols:
    env_ratings[col] = env_ratings[col].map(names)

env_ratings = env_ratings.set_index(['model', 'decile', 'sent', 'sort'])
env_ratings = env_ratings.drop(columns='name')

env_ratings.index.name = None


# format coverage as %
env_ratings['emissions:coverage'] = env_ratings['emissions:coverage'].apply(lambda x: f'{x:.0%}')

#env_ratings

In [ ]:
env_ratings = env_ratings.transpose()
env_ratings['names'] = env_ratings.index
env_ratings['model'] = env_ratings['names'].str.split(':', expand=True)[0]
env_ratings['stat'] = env_ratings['names'].str.split(':', expand=True)[1]
env_ratings = env_ratings.set_index(['model', 'stat'], drop=True)
env_ratings = env_ratings.drop(columns='names')
env_ratings = env_ratings.transpose()
env_ratings

In [ ]:
latex = env_ratings.to_latex(
    index=True,
    escape=True,
    sparsify=True,
    multirow=True,
    multicolumn=True,
    bold_rows=True,
    multicolumn_format="c",
    float_format="{:.2f}".format,
    position="H",
)

latex = re.sub(r"\\(mid|top|bottom)rule", "", latex)

print(latex)


In [ ]:
def top_bottom(df):
    top10 = (
        df["snp_ric"]
        .to_frame()
        .apply(pd.Series.value_counts)
        .sort_values(by="snp_ric", ascending=False)
        .head(10)
    )

    top10 = pd.merge(
        left=top10,
        right=df.groupby("snp_ric")["env"].mean().to_frame(),
        left_index=True,
        right_index=True,
    )

    top10 = pd.merge(
        left=top10,
        right=df.groupby("snp_ric")["emissions"].mean().to_frame(),
        left_index=True,
        right_index=True,
    )

    top10 = pd.merge(
        left=top10,
        right=meta[["company_common_name", "trbc_industry_name"]],
        left_index=True,
        right_index=True,
    )

    top10 = top10.rename(
        columns={
            "env": "Mean E Rating",
            "snp_ric": "Occurrences",
            "company_common_name": "Company Name",
            "trbc_industry_name": "Industry",
            'emissions': 'CO2'
        }
    )

    # move index
    top10["Ticker"] = top10.index
    top10 = top10.reset_index(drop=True)

    top10 = top10[
        ["Ticker", "Company Name", "Industry", "Occurrences", "Mean E Rating", 'CO2']
    ]

    return top10

# Importance of human...

In [ ]:
top_10_sorts = top_bottom(sorts["bottom:0.2:CAPM:importance_of_human_intervantion_transformed_residuals"].copy())
bottom_10_sorts = top_bottom(sorts["top:0.2:CAPM:importance_of_human_intervantion_transformed_residuals"].copy())


top_10_sorts['Sort'] = 'Top 10'
bottom_10_sorts['Sort'] = 'Bottom 10'

summary_importance = pd.concat([top_10_sorts, bottom_10_sorts])

summary_importance = summary_importance.set_index('Sort')

summary_importance = summary_importance.drop(columns=['Ticker'])

summary_importance

In [ ]:
latex = summary_importance.to_latex(
    index=True,
    escape=True,
    sparsify=True,
    multirow=True,
    multicolumn=True,
    bold_rows=True,
    multicolumn_format="c",
    float_format="{:.2f}".format,
    position="H",
)

latex = re.sub(r"\\(mid|top|bottom)rule", "", latex)

print(latex)


# Politics...

In [ ]:
top_10_sorts = top_bottom(sorts["bottom:0.2:FF3:politics_transformed_residuals"].copy())
bottom_10_sorts = top_bottom(sorts["top:0.2:FF3:politics_transformed_residuals"].copy())


top_10_sorts['Sort'] = 'Top 10'
bottom_10_sorts['Sort'] = 'Bottom 10'

summary_pol = pd.concat([top_10_sorts, bottom_10_sorts])

summary_pol = summary_pol.set_index('Sort')

summary_pol = summary_pol.drop(columns=['Ticker'])

summary_pol

In [ ]:
mean = summary_pol.loc[summary_pol['Company Name'] != 'Southern Co'].dropna()['CO2'].groupby('Sort').mean()

std = summary_pol.loc[summary_pol['Company Name'] != 'Southern Co'].dropna()['CO2'].groupby('Sort').std()

display(mean)

In [ ]:
latex = summary_pol.to_latex(
    index=True,
    escape=True,
    sparsify=True,
    multirow=True,
    multicolumn=True,
    bold_rows=True,
    multicolumn_format="c",
    float_format="{:.2f}".format,
    position="H",
)

latex = re.sub(r"\\(mid|top|bottom)rule", "", latex)

print(latex)


In [ ]:
names

In [ ]:
hml_corr = factors['e-hml'].dropna().to_frame()

for key, df in estimated_portfolio_sorts.items():
    if key not in significant_models:
        continue

    #display(key)
    #display(df.groupby('date')['hml_return'].first())
    hml_corr = pd.merge(left=hml_corr, right=df.groupby('date')['hml_return'].first(), left_index=True, right_index=True)
    hml_corr = hml_corr.rename(columns={'hml_return': key})


hml_corr = hml_corr.transpose()

hml_corr['names'] = hml_corr.index

hml_corr['sort'] = [names.get(i) for i in hml_corr['names'].str.split(':', expand=True)[0]]
hml_corr['model'] = [names.get(i) for i in hml_corr['names'].str.split(':', expand=True)[1]]
hml_corr['sent'] = [names.get(i) for i in hml_corr['names'].str.split(':', expand=True)[2]]

hml_corr[['sort', 'model', 'sent']] = hml_corr[['sort', 'model', 'sent']].fillna('e-hml')
hml_corr = hml_corr.set_index(['model', 'sort', 'sent'])

hml_corr = hml_corr.drop(columns='names').transpose()

In [ ]:
mask = np.triu(np.ones_like(hml_corr.corr()))
np.fill_diagonal(mask, 0)

fig, ax = plt.subplots(figsize=(14,5))

sns.heatmap(hml_corr.corr(), annot=True, fmt=".2f", ax=ax, linewidths=0.5, mask=mask)
ax.set(xlabel="", ylabel="")

labels = [item.get_text() for item in ax.get_xticklabels()]
labels[0] = 'e-hml'

ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

plt.savefig(f"plots/correlation_matrix.png", dpi=450, bbox_inches="tight")
